In [4]:
import pandas as pd
import numpy as np
import pandas_profiling as pd_pro
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import gc
import lightgbm as lgbm
from catboost import CatBoostRegressor

In [5]:
test = pd.read_csv('input/test.csv', parse_dates=['ClickDate'], 
                    dtype= { 'ID' : np.int32, 
                            'Carrier' : np.float32,                             
                             #'publisherId' : np.int32,
                            'advertiserCampaignId' : np.float32,
                            'Fraud': np.float32},
                     encoding='UTF-8')  
                   

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
test_Id = test['ID'].values

In [7]:
#chunksize = 10 ** 7
#chunks = pd.read_csv('input/train.csv', chunksize=chunksize)
train = pd.read_csv('input/train.csv', parse_dates=['ClickDate'], 
                    dtype= { 'ID' : np.int32, 
                            'Carrier' : np.float32, 
                            'ConversionPayOut' : np.float32,
                             #'publisherId' : np.int32,
                            'advertiserCampaignId' : np.float32,
                            'Fraud': np.float32},
                     encoding='UTF-8',  
                     nrows= 15000000)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
len_train = train.shape[0]
y_train = train.ConversionPayOut.values
train.drop(['ConversionStatus','ConversionDate', 'ConversionPayOut'], inplace = True, axis=1)

In [9]:
data = pd.concat([train, test])

In [10]:
del train, test
gc.collect()

942

In [11]:
data.Country.fillna('other', inplace = True)
data.TrafficType.fillna('other', inplace = True)
data.Device.fillna('other', inplace = True)
data.Browser.fillna('other', inplace = True)
data.OS.fillna('other', inplace = True)
data.RefererUrl.fillna('other', inplace = True)
data.subPublisherId.fillna('other', inplace = True)
data['publisherId'] = pd.to_numeric(data['publisherId'], errors='coerce')
data['subPublisherId'] = pd.to_numeric(data['subPublisherId'], errors='coerce')

In [12]:
cols_to_encode = ['Country',  'TrafficType', 'Device','Browser', 'OS', 'RefererUrl', 'UserIp', 
                 'subPublisherId','publisherId']
le = LabelEncoder()
for col in cols_to_encode:
    data[col] = le.fit_transform(data[col])
    

In [13]:
data['tweekday'] = data['ClickDate'].dt.weekday
data['thour'] = data['ClickDate'].dt.hour
data['tminute'] = data['ClickDate'].dt.minute

In [14]:
#data.head(5)

In [15]:
ref_url_cnt = data.groupby(['UserIp'])[ 'RefererUrl'].nunique().reset_index().rename(columns = {'RefererUrl': 'ref_url_cnt'})

In [16]:
pubId_cnt = data.groupby(['UserIp'])[ 'publisherId'].nunique().reset_index().rename(columns = {'publisherId': 'pubId_cnt'})

In [17]:
adv_comp_cnt = data.groupby(['UserIp'])[ 'advertiserCampaignId'].nunique().reset_index().\
                             rename(columns = {'advertiserCampaignId': 'compId_cnt'})

In [18]:
data = pd.merge(data, ref_url_cnt, how='left', on=['UserIp'])
data = pd.merge(data, pubId_cnt, how='left', on=['UserIp'])
data = pd.merge(data, adv_comp_cnt, how='left', on=['UserIp'])

In [19]:
del ref_url_cnt, pubId_cnt, adv_comp_cnt
gc.collect()

63

In [20]:
data['refUrl_user_cnt'] = data.groupby(['RefererUrl'])['ID'].transform('count')

In [21]:
data['user_cnt'] = data.groupby(['UserIp'])['ID'].transform('count')

In [22]:
data['user_week_cnt'] = data.groupby(['UserIp', 'tweekday'])['ID'].transform('count')
data['user_hr_cnt'] = data.groupby(['UserIp', 'tweekday', 'thour'])['ID'].transform('count')

In [23]:
data['user_refurl_share'] = data['user_cnt']/ data['ref_url_cnt']
data['user_pubid_share'] = data['user_cnt']/ data['pubId_cnt']
data['user_compid_share'] = data['user_cnt']/data['compId_cnt']

In [24]:
data.drop(['ID','ClickDate'], axis=1, inplace=True)
train = data[:len_train]
test = data[len_train:]
test.shape

(25548873, 25)

In [25]:
del data
gc.collect()

70

In [31]:
train_X, valid_X, train_y, valid_y = train_test_split(train, y_train, test_size=0.3, random_state=32)

In [32]:
#print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape)

In [33]:
lgb_train = lgbm.Dataset(train_X, train_y)
lgb_eval = lgbm.Dataset(valid_X, valid_y, reference=lgb_train)

In [34]:
del train_X, valid_X, train_y, valid_y

In [46]:
params1 = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 25,
    'min_data_in_leaf':40,
    'max_bin':10,
    'learning_rate': 0.002,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    #'device':'gpu',
    'verbose': 0,
    'seed': 32
}

In [36]:
model_gbm = lgbm.train(params1,
                lgb_train,                
                num_boost_round=500,
                valid_sets=lgb_eval,
                early_stopping_rounds=100)

[1]	valid_0's rmse: 5.52184
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 5.52184
[3]	valid_0's rmse: 5.52184
[4]	valid_0's rmse: 5.52184
[5]	valid_0's rmse: 5.52184
[6]	valid_0's rmse: 5.52184
[7]	valid_0's rmse: 5.52184
[8]	valid_0's rmse: 5.52184
[9]	valid_0's rmse: 5.52184
[10]	valid_0's rmse: 5.52184
[11]	valid_0's rmse: 5.52184
[12]	valid_0's rmse: 5.52184
[13]	valid_0's rmse: 5.52184
[14]	valid_0's rmse: 5.52184
[15]	valid_0's rmse: 5.52184
[16]	valid_0's rmse: 5.52184
[17]	valid_0's rmse: 5.52184
[18]	valid_0's rmse: 5.52184
[19]	valid_0's rmse: 5.52184
[20]	valid_0's rmse: 5.52184
[21]	valid_0's rmse: 5.52184
[22]	valid_0's rmse: 5.52184
[23]	valid_0's rmse: 5.52184
[24]	valid_0's rmse: 5.52184
[25]	valid_0's rmse: 5.52184
[26]	valid_0's rmse: 5.52184
[27]	valid_0's rmse: 5.52184
[28]	valid_0's rmse: 5.52185
[29]	valid_0's rmse: 5.52185
[30]	valid_0's rmse: 5.52185
[31]	valid_0's rmse: 5.52185
[32]	valid_0's rmse: 5.52185
[33]	valid_0's rms

In [37]:
del lgb_train, lgb_eval
gc.collect()

127

In [38]:
train_X, valid_X, train_y, valid_y = train_test_split(train, y_train, test_size=0.23, random_state=13)

In [39]:
lgb_train = lgbm.Dataset(train_X, train_y)
lgb_eval = lgbm.Dataset(valid_X, valid_y, reference=lgb_train)

In [40]:
del train_X, valid_X, train_y, valid_y

In [44]:
params2 = {
    'task': 'train',
    'boosting_type': 'rf',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 25,
    'min_data_in_leaf':40,
    'max_bin':10,
    'learning_rate': 0.002,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    #'device':'gpu',
    'verbose': 0,
    'seed': 13
}

In [42]:
model_rf = lgbm.train(params2,
                lgb_train,                
                num_boost_round=500,
                valid_sets=lgb_eval,
                early_stopping_rounds=100)

[1]	valid_0's rmse: 5.66433
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 5.66433
[3]	valid_0's rmse: 5.66249
[4]	valid_0's rmse: 5.66226
[5]	valid_0's rmse: 5.66092
[6]	valid_0's rmse: 5.65947
[7]	valid_0's rmse: 5.65858
[8]	valid_0's rmse: 5.65859
[9]	valid_0's rmse: 5.65836
[10]	valid_0's rmse: 5.65827
[11]	valid_0's rmse: 5.65806
[12]	valid_0's rmse: 5.65809
[13]	valid_0's rmse: 5.65799
[14]	valid_0's rmse: 5.65788
[15]	valid_0's rmse: 5.65799
[16]	valid_0's rmse: 5.65776
[17]	valid_0's rmse: 5.65758
[18]	valid_0's rmse: 5.6573
[19]	valid_0's rmse: 5.65723
[20]	valid_0's rmse: 5.65715
[21]	valid_0's rmse: 5.65704
[22]	valid_0's rmse: 5.65695
[23]	valid_0's rmse: 5.65676
[24]	valid_0's rmse: 5.65667
[25]	valid_0's rmse: 5.65668
[26]	valid_0's rmse: 5.65679
[27]	valid_0's rmse: 5.6569
[28]	valid_0's rmse: 5.657
[29]	valid_0's rmse: 5.65694
[30]	valid_0's rmse: 5.65706
[31]	valid_0's rmse: 5.65699
[32]	valid_0's rmse: 5.65697
[33]	valid_0's rmse: 5

[277]	valid_0's rmse: 5.65585
[278]	valid_0's rmse: 5.65584
[279]	valid_0's rmse: 5.65584
[280]	valid_0's rmse: 5.65585
[281]	valid_0's rmse: 5.65585
[282]	valid_0's rmse: 5.65586
[283]	valid_0's rmse: 5.65587
[284]	valid_0's rmse: 5.65587
[285]	valid_0's rmse: 5.65588
[286]	valid_0's rmse: 5.65589
[287]	valid_0's rmse: 5.65588
[288]	valid_0's rmse: 5.65589
[289]	valid_0's rmse: 5.65589
[290]	valid_0's rmse: 5.6559
[291]	valid_0's rmse: 5.65589
[292]	valid_0's rmse: 5.65589
[293]	valid_0's rmse: 5.65589
[294]	valid_0's rmse: 5.65588
[295]	valid_0's rmse: 5.65588
[296]	valid_0's rmse: 5.65589
[297]	valid_0's rmse: 5.65589
[298]	valid_0's rmse: 5.65589
[299]	valid_0's rmse: 5.6559
[300]	valid_0's rmse: 5.65591
[301]	valid_0's rmse: 5.65591
[302]	valid_0's rmse: 5.65591
[303]	valid_0's rmse: 5.65591
[304]	valid_0's rmse: 5.6559
[305]	valid_0's rmse: 5.6559
[306]	valid_0's rmse: 5.65591
[307]	valid_0's rmse: 5.65591
[308]	valid_0's rmse: 5.65592
[309]	valid_0's rmse: 5.65592
[310]	valid_0'

In [50]:
train_X, valid_X, train_y, valid_y = train_test_split(train, y_train, test_size=0.27, random_state=4)

In [51]:
model_cat = CatBoostRegressor(depth=8, iterations=200, learning_rate=0.03, eval_metric='RMSE', random_seed=1)
model_cat.fit(train_X
          ,train_y          
          ,eval_set = (valid_X, valid_y)
          ,use_best_model = True
         )

In [52]:
del train_X, valid_X, train_y, valid_y
gc.collect()

1047

In [53]:
y_pred_gbm = model_gbm.predict(test, num_iteration=model_gbm.best_iteration)

In [54]:
y_pred_rf = model_rf.predict(test, num_iteration=model_gbm.best_iteration)

In [55]:
y_pred_cat = model_cat.predict(test)

In [56]:
y_pred = (y_pred_gbm + y_pred_rf + y_pred_cat)/3.0

In [57]:

del train, test
gc.collect()

70

In [59]:
y_pred[:10]

array([ 0.02064026,  0.0382833 ,  0.02169453,  0.01336993,  0.01595702,
        0.01457948,  0.01732141,  0.01490231,  0.01897298,  0.01819507])

In [65]:
sub = pd.DataFrame({'ID': test_Id, 'ConversionPayOut' : y_pred_cat })
sub = sub[['ID','ConversionPayOut'] ]

In [66]:
sub.head(5)

,ID,ConversionPayOut
0,63367289,0.004721
1,63367290,0.079569
2,63367291,0.007884
3,63367292,0.003268
4,63367293,0.010227


In [67]:
sub.to_csv('cat_v1.csv', index=False)

In [64]:
del model_gbm, model_rf, model_cat
gc.collect()

105